##### LINK DE REPOSITORIO
https://github.com/FabianKel/LAB2-IA

# **TASK #1 - Preguntas Teóricas**

### ¿Por qué el modelo de Naive Bayes se le considera “naive”?

- El modelo se considera naive debido a que asume que las características de medición son independientes entre sí y contribuyen por igual en el momento de dar resultado. Esta suposición de independencia es rara vez es cierta. Sin embargo, permite que el modelo sea muy eficiente en términos de tiempo de computación y memoria.

### Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo? (Lo que se espera de esta pregunta es que puedan explicar en sus propias palabras la fórmula a la que llegamos que debemos optimizar de SVM en clase)

- La formulación matemática que se busca optimizar en Support Vector Machine (SVM) es maximizar el margen entre las clases o bien como fue mencionado en clase crear una frontera para separar las diferentes clases con las que estamos logrando. Esto se logra resolviendo el problema de optimización cuadrática donde se aseguran que los datos estén correctamente clasificados y fuera del margen. La función es:

$$\min_{(\mathbf{w}, b)} \frac{1}{2} \|\mathbf{w}\|^2$$

sujeto a:

$$y_i (\mathbf{w} \cdot \mathbf{x}_i + b) \geq 1 \quad \forall i$$

donde $\mathbf{w}$ es el vector de pesos, $b$ es el sesgo, $\mathbf{x}_i$ son los vectores de características y $y_i$ son las etiquetas de clase.

- El truco del Kernel permite a SVM manejar datos no linealmente separables al transformar los datos originales a un espacio de mayor dimensión donde es más probable que sean linealmente separables. Esto se hace utilizando una función de kernel \(K(\mathbf{x}_i, \mathbf{x}_j)\) que va calcula el producto punto en el espacio transformado sin necesidad de calcular explícitamente la transformación.
### Investigue sobre Random Forest y responda
1. ¿Qué tipo de ensemble learning es este modelo?
    - El tipo de ensemble learning es Bagging (Bootstrap Aggregating).

2. ¿Cuál es la idea general detrás de Random Forest?
    - La idea general detrás de Random Forest es construir múltiples árboles de decisión durante el entrenamiento y clasificación o la media de las predicciones (regresión) de los árboles individuales. Cada árbol es entrenado con una muestra aleatoria del dataset original con reemplazo (bootstrap sample), y en cada nodo de los árboles, solo un subconjunto aleatorio de características es considerado para la división.

3. ¿Por qué se busca baja correlación entre los árboles de Random Forest?
    - Se busca baja correlación entre los árboles de Random Forest porque si los árboles son altamente correlacionados, sus errores también serán correlacionados, lo que reduce la efectividad del ensemble. La baja correlación asegura que los errores de los árboles individuales se cancelen entre sí, mejorando la precisión y robustez del modelo final.


# **TASK #2 - Naive Bayes: Clasificador de Mensajes Ham/Spam**

Importación de librerias

In [8]:
import numpy as np
from collections import defaultdict
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

Importación de Archivo a Utilizar 

In [9]:
def load_dataset(filename):
    messages = []
    labels = []
    
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            label, message = line.strip().split('\t', 1)
            messages.append(message)
            labels.append(label)
    
    return messages, labels

# Cargar el dataset
messages, labels = load_dataset('entrenamiento.txt')

print("Primeros 15 mensajes del dataset:")
for msg, label in zip(messages[:15], labels[:15]):
    print(f"{label}: {msg}\n")

Primeros 15 mensajes del dataset:
ham: Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

ham: Ok lar... Joking wif u oni...

spam: Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

ham: U dun say so early hor... U c already then say...

ham: Nah I don't think he goes to usf, he lives around here though

spam: FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv

ham: Even my brother is not like to speak with me. They treat me like aids patent.

ham: As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune

spam: WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 0906170

Preprocesar Texto

In [12]:
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenización simple por espacios
    return text.split()

# Ejemplo la función de preprocesamiento
texto = messages[0]
print("Texto original:", texto)
print("Texto procesado:", preprocess_text(texto))

Texto original: Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Texto procesado: ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']


Dataset dividido en training y test

In [14]:
# Training y Test
X_train, X_test, y_train, y_test = train_test_split(
    messages, labels, test_size=0.2, random_state=42
)

print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de prueba: {len(X_test)}")

Tamaño del conjunto de entrenamiento: 4452
Tamaño del conjunto de prueba: 1113


In [17]:
def TrainNaive_bayes(messages, labels, alpha=1.0):

    # contadores
    WordCounts = defaultdict(lambda: defaultdict(int))
    ClassCounts = defaultdict(int)
    Vocabulary = set()
    
    # Contar ocurrencias de palabras y clases
    for message, label in zip(messages, labels):
        ClassCounts[label] += 1
        words = preprocess_text(message)
        
        for word in words:
            WordCounts[label][word] += 1
            Vocabulary.add(word)
    
    # Calcular probabilidades de clase
    TotalDocs = len(messages)
    ClassProbs = {
        label: count/TotalDocs 
        for label, count in ClassCounts.items()
    }
    
    # Calcular probabilidades de palabras con suavizado de Laplace
    vocab_size = len(Vocabulary)
    word_probs = {}
    
    for label in ClassCounts.keys():
        total_words = sum(WordCounts[label].values())
        word_probs[label] = {}
        
        for word in Vocabulary:
            numerator = WordCounts[label][word] + alpha
            denominator = total_words + alpha * vocab_size
            word_probs[label][word] = numerator / denominator
    
    return word_probs, ClassProbs, Vocabulary

# Entrenamos el modelo
word_probs, ClassProbs, Vocabulary = TrainNaive_bayes(X_train, y_train)

print("Probabilidades de clase:", ClassProbs)
print("\nTamaño del vocabulario:", len(Vocabulary))

Probabilidades de clase: {'ham': 0.8688230008984726, 'spam': 0.13117699910152741}

Tamaño del vocabulario: 7661


# **TASK #3 - Clasificación de Partidas de League of Legends**

## **Referencias**

¿Qué es Support Vector Machine? (2024, octubre 4). Ibm.com. https://www.ibm.com/mx-es/topics/support-vector-machine

susmit_sekhar_bhakta Follow Improve. (2024, febrero 22). Random Forest algorithm in machine learning. GeeksforGeeks. https://www.geeksforgeeks.org/random-forest-algorithm-in-machine-learning/

What is random forest? (2024, diciembre 19). Ibm.com. https://www.ibm.com/think/topics/random-forest